In [150]:
# Packages used in this notebook
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# setting the starting url 
url_overview_köln = "https://www.wg-gesucht.de/wg-zimmer-in-Koeln.73.0.0.1.html"

In [151]:
# Testing 
# Sends a GET request to the URL to retrieve its content; The Website tryes to redicrect the request to a captcha page, reloading solves the problem
response = requests.get(url_overview_köln, allow_redirects=False)
# assert response.status_code == 200
print(response.status_code)
# Parse results
soup = BeautifulSoup(response.text, 'html.parser')

target_class = "ang_spalte_icons row_click"
elements = soup.find_all('td', class_=target_class)
links=[]
for element in elements:
    links.append(element.find('a').get('href'))
# Extracting all shared apartments
#for link in soup.find_all('a'):
#    print(link.get('href'))
print(links)
print(len(links))


200
['wg-zimmer-in-Koeln-Nippes.3577779.html', 'wg-zimmer-in-Koeln.10997471.html', 'wg-zimmer-in-Koeln-Loevenich.11017719.html', 'wg-zimmer-in-Koeln-Raderberg.11017983.html', 'wg-zimmer-in-Koeln-Bickendorf.11018021.html', 'wg-zimmer-in-Koeln-Humboldt-Gremberg.11015946.html', 'wg-zimmer-in-Koeln-Weiden.10995835.html', 'wg-zimmer-in-Koeln-Junkersdorf.11017749.html', 'wg-zimmer-in-Koeln-Ehrenfeld.10967178.html', 'wg-zimmer-in-Koeln-Neuehrenfeld-Neu-Ossendorf.11017025.html', 'wg-zimmer-in-Koeln-Bocklemuend-Mengenich.7226799.html', 'wg-zimmer-in-Koeln-Kalk.11017437.html', 'wg-zimmer-in-Koeln.11007372.html', 'wg-zimmer-in-Koeln-Brueck.10036288.html', 'wg-zimmer-in-Koeln-Agnesviertel.10188646.html', 'wg-zimmer-in-Koeln-Koeln.10953730.html', 'wg-zimmer-in-Koeln-Ehrenfeld.6654704.html', 'wg-zimmer-in-Koeln-Ehrenfeld.8086704.html', 'wg-zimmer-in-Koeln-Nippes.9100527.html', 'wg-zimmer-in-Koeln-Altstadt-Sued.10081212.html']
20


302 => Captcha -->nochmal probieren
200 => Okay

wnen die Klasse auf einer Seite nicht mehr gefunden werden kann, enthält diese keine Angebote mehr

In [152]:
page = 0
failed = 0
base_links=[]
campaign_links=[]

# main class for the links
target_class = "truncate_title noprint"

# class for the campaign links
campaign_class = "campaign_click"

last_page = page
cum_fails = 0
while page <= 20:
    # setting the url 
    url_overview_köln = "https://www.wg-gesucht.de/wg-zimmer-in-Koeln.73.0.1."+str(page)+".html"

    # Sends a GET request to the URL to retrieve its content; sometimes the Website tryes to redicrect the request to a captcha page, reloading solves the problem
    response = requests.get(url_overview_köln, allow_redirects=False)
    
    # test response code
    if response.status_code == 302 and failed < 5:
        # try again
        failed += 1 
        cum_fails += 1
        continue
    elif failed >= 5:
        print("Failed 5 times in a row")
        break
    
    page += 1
    failed = 0

    # Parse results
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extracting all shared apartments
    elements = soup.find_all('h3', class_ = target_class)

    if len(elements) == 0:
        print("No more elements found")
        break

    
    for element in elements:
        for appartment in element.find_all('a', class_ = campaign_class):
            campaign_links.append(element.find('a').get('href'))
        for appartment in element.find_all('a', class_=''):
            base_links.append(element.find('a').get('href'))

    # just for debugging
    last_page = page -1


# TODO: aufhübschen und in Funktionen packen
print("Base links: ")
print(base_links)

set_base_links = set(base_links)
base_links = list(set_base_links)
print("Anzahl der Base links:" + str(len(base_links)))



print("Campaign links: " )
print(campaign_links)

set_campaign_links = set(campaign_links)
campaign_links = list(set_campaign_links)
print("Anzahl der Campaign links:" + str(len(campaign_links)))
print("Pages: "+str(last_page))
print("Failed: "+str(cum_fails))


Base links: 
['/wg-zimmer-in-Koeln-Kalk.10986900.html', '/wg-zimmer-in-Koeln-Kalk.9510267.html', '/wg-zimmer-in-Koeln-Altstadt-Sued.7022036.html', '/wg-zimmer-in-Koeln.10989742.html', '/wg-zimmer-in-Koeln-Ehrenfeld.11018175.html', '/wg-zimmer-in-Koeln-Altstadt-Sued.10954009.html', '/wg-zimmer-in-Koeln-Deutz-Muelheim.9996498.html', '/wg-zimmer-in-Koeln.11018160.html', '/wg-zimmer-in-Koeln-Marienburg.3216416.html', '/wg-zimmer-in-Koeln-Klettenberg.8996879.html', '/wg-zimmer-in-Koeln-Vingst.11018132.html', '/wg-zimmer-in-Koeln-Vingst.11018107.html', '/wg-zimmer-in-Koeln-Riehl.4764812.html', '/wg-zimmer-in-Koeln-Lindenthal.5313349.html', '/wg-zimmer-in-Koeln-Bornheim.10900587.html', '/wg-zimmer-in-Koeln-Zollstock.10138614.html', '/wg-zimmer-in-Koeln-Marienburg.10590012.html', '/wg-zimmer-in-Koeln-Altstadt-Sued.6068399.html', '/wg-zimmer-in-Koeln-West.6980083.html', '/wg-zimmer-in-Koeln.11017740.html', '/wg-zimmer-in-Koeln-Nippes.3577779.html', '/wg-zimmer-in-Koeln.10997471.html', '/wg-zimm

nur verifizierte Unternehmen haben die Klasse "campaign clicks" --> sind auch unten als Werbung aufgeführt

verfifizierte Personen nicht



416 Ergebnisse sind okay. Erwartet hätten wir 420 -> ggf. Werbungen?

In [153]:
# TODO: remove wrong links in the camoaign links (asset_id...)



In [257]:
# TODO: für eine Einzelseite die Attribute bestimmen
result_df = pd.DataFrame(columns=['Link', 'Title', 'Size', 'Rent', 'Extra Costs', 'Other Costs', 'Deposit', 'Redemption Agreement', 'Address', 'Available From', 'Available Till', 'Online Since'])

url_shared_app = "https://www.wg-gesucht.de" + str(base_links[0]) 
response = requests.get(url_shared_app)

soup = BeautifulSoup(response.text, 'html.parser')

# Extracting the title
title = soup.find('h1', class_='headline headline-detailed-view-title').text
title = title.replace('\n', '').strip()

# Extracting the size
size = soup.find('b', class_='key_fact_value').text
size = size.replace('\n', '').strip()

# TODO: Adding the count of people in the flat
# TODO: Maybe Restructure the replacing of the \n and strip to a function and do it in the df
# Extracting the cost panel
cost_panel = soup.find_all('div', class_='panel section_panel')[1]

rent = cost_panel.find_all('span', class_='section_panel_value')[0].text
rent = rent.replace('\n', '').strip()

extra_costs = cost_panel.find_all('span', class_='section_panel_value')[1].text
extra_costs = extra_costs.replace('\n', '').strip()

other_costs = cost_panel.find_all('span', class_='section_panel_value')[2].text
other_costs = other_costs.replace('\n', '').strip()

deposit = cost_panel.find_all('span', class_='section_panel_value')[3].text
deposit = deposit.replace('\n', '').strip()

redemption_agreement = cost_panel.find_all('span', class_='section_panel_value')[4].text
redemption_agreement = redemption_agreement.replace('\n', '').strip()



# Address and Availability Panel
address_panel = soup.find_all('div', class_='panel section_panel')[2]

address = address_panel.find_all('span', class_='section_panel_detail')[0].text
address = ' '.join(address.split())

available_from = address_panel.find_all('span', class_='section_panel_value')[0].text
available_from = available_from.replace('\n', '').strip()

# Sometimes the available till is not available, so we need to check if the second span is 'frei bis:' --> if not we need to set it to n.a.
if soup.find_all('span', class_='section_panel_detail')[1].text == 'frei bis:':
    available_till = address_panel.find_all('span', class_='section_panel_value')[1].text
    available_till = available_till.replace('\n', '').strip()
    online_since = address_panel.find_all('span', class_='section_panel_value')[2].text
    online_since = online_since.replace('\n', '').strip()

else:
    available_till = "n.a."
    online_since = address_panel.find_all('b', class_='noprint')[0].text
    online_since = online_since.replace('\n', '').strip()


input_set = {'Link': url_shared_app, 'Title': title, 'Size': size, 'Rent': rent, 'Extra Costs': extra_costs, 'Other Costs': other_costs, 'Deposit': deposit, 'Redemption Agreement': redemption_agreement, 'Address': address, 'Available From': available_from, 'Available Till': available_till, 'Online Since': online_since}
result_df = result_df._append(input_set, ignore_index=True)

result_df

15.06.2024


,Link,Title,Size,Rent,Extra Costs,Other Costs,Deposit,Redemption Agreement,Address,Available From,Available Till,Online Since
0,https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Z...,"1 Zimmer in eine WG (Zollstock, Köln)",23m²,550€,n.a.,n.a.,n.a.,n.a.,An der Kranzmaar 50968 Köln Zollstock,15.06.2024,n.a.,4 Tage
